In [0]:
# Import spark session
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
# Importing regression algorithm
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [0]:
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[10]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[11]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary

In [0]:
training_summary.rootMeanSquaredError

Out[18]: 10.16309157133015

In [0]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
# Splitting the data into training & testing
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                367|
|   mean|-0.1109784323589348|
| stddev| 10.217557717863965|
|    min|-28.571478869743427|
|    max| 22.647750304177556|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                134|
|   mean| 1.2644058551896886|
| stddev| 10.560898502940521|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
test_results.rootMeanSquaredError


Out[30]: 10.810396940998476

In [0]:
# Model Deployment
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data) # Prediction using transform
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.3770478832137717|
|(10,[0,1,2,3,4,5,...|   0.752689830776103|
|(10,[0,1,2,3,4,5,...|-0.04888649250754376|
|(10,[0,1,2,3,4,5,...|   2.733743758945588|
|(10,[0,1,2,3,4,5,...|  3.2406272056635443|
|(10,[0,1,2,3,4,5,...|  1.1027259373918266|
|(10,[0,1,2,3,4,5,...|   4.431684453464886|
|(10,[0,1,2,3,4,5,...| -3.4940382518246724|
|(10,[0,1,2,3,4,5,...| -1.0665522169906734|
|(10,[0,1,2,3,4,5,...|  -2.692801553917049|
|(10,[0,1,2,3,4,5,...|   2.279678259243184|
|(10,[0,1,2,3,4,5,...| -2.4073617402444656|
|(10,[0,1,2,3,4,5,...|   2.519686270440036|
|(10,[0,1,2,3,4,5,...|-0.23902167026563606|
|(10,[0,1,2,3,4,5,...| -1.6043529745463123|
|(10,[0,1,2,3,4,5,...| -2.9883229851424358|
|(10,[0,1,2,3,4,5,...|  -1.718701687445971|
|(10,[0,1,2,3,4,5,...|  -2.679462745832818|
|(10,[0,1,2,3,4,5,...| -0.7451615626334406|
|(10,[0,1,2,3,4,5,...|  0.094617